In [1]:
! pip install pyarrow


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import pandas as pd
import pyarrow.parquet as pq
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import os

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, root_mean_squared_error

In [3]:
def load_data(url_parquet):
    url = url_parquet
    file_name = url.split('/')[-1]
    os.system(f'curl -o data/{file_name} "{url}"')
    print("Data downloaded")

In [14]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    print(len(df.columns))

    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    print(df.duration.describe().round(2))

    print(df[(df.duration >= 1) & (df.duration <= 60)].shape[0] / df.shape[0])

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [7]:
url_train = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet'
url_val = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet'
load_data(url_train)
load_data(url_val)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.4M  100 45.4M    0     0   355M      0 --:--:-- --:--:-- --:--:--  355M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

Data downloaded
Data downloaded


100 45.5M  100 45.5M    0     0   392M      0 --:--:-- --:--:-- --:--:--  392M


In [15]:
df_train = read_dataframe('./data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2023-02.parquet')

19
count    3066766.00
mean          15.67
std           42.59
min          -29.20
25%            7.12
50%           11.52
75%           18.30
max        10029.18
Name: duration, dtype: float64
0.9812202822125979
19
count    2913955.00
mean          16.02
std           42.84
min          -43.62
25%            7.25
50%           11.80
75%           18.77
max         7053.62
Name: duration, dtype: float64
0.9800944077722545


In [16]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

print(X_train.shape)

(3009173, 515)


In [17]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [18]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
y_pred_t = lr.predict(X_train)

print("train MSE: ", mean_squared_error(y_train, y_pred_t))
print("train MAE: ", mean_absolute_error(y_train, y_pred_t))
print("train RMSE: ", root_mean_squared_error(y_train, y_pred_t))


print("val MSE: ", mean_squared_error(y_val, y_pred))
print("val MAE: ", mean_absolute_error(y_val, y_pred))
print("val RMSE: ", root_mean_squared_error(y_val, y_pred))

train MSE:  58.51121587152861
train MAE:  5.710800794524329
train RMSE:  7.6492624397080675
val MSE:  61.0244085027562
val MAE:  5.821991682902076
val RMSE:  7.81181211389241
